In [1]:
import os
import shutil
import random
from pathlib import Path

# Kaynak klasörler
source_dirs = {
    "real": [r"C:\Users\Admin\Desktop\TEZ\Deepfake\Celeb-real", r"C:\Users\Admin\Desktop\TEZ\Deepfake\youtube-real"],
    "fake": [r"C:\Users\Admin\Desktop\TEZ\Deepfake\Celeb-synthesis"]
}

# Hedef kök klasör
output_root = r"C:\Users\Admin\Desktop\TEZ\Deepfake"
train_ratio = 0.8  # %80 eğitim, %20 test

# Sabit tutmak için random seed
random.seed(42)

# Hedef klasörleri oluştur
for split in ["train", "test"]:
    for cls in ["real", "fake"]:
        Path(f"{output_root}/{split}/{cls}").mkdir(parents=True, exist_ok=True)

# Tüm videoları topla ve sınıflandır
video_paths = {"real": [], "fake": []}

for label, dirs in source_dirs.items():
    for folder in dirs:
        for video in Path(folder).glob("*.*"):
            if video.suffix.lower() in [".mp4", ".avi", ".mov", ".mkv"]:  # yaygın video formatları
                video_paths[label].append(video)

# Eğitim ve test olarak ayır, sonra kopyala
for label, paths in video_paths.items():
    random.shuffle(paths)
    split_index = int(len(paths) * train_ratio)
    train_videos = paths[:split_index]
    test_videos = paths[split_index:]

    # Kopyala
    for video in train_videos:
        shutil.copy(video, f"{output_root}/train/{label}/{video.name}")
    for video in test_videos:
        shutil.copy(video, f"{output_root}/test/{label}/{video.name}")

print("✅ Videolar başarıyla train/test klasörlerine kopyalandı.")


✅ Videolar başarıyla train/test klasörlerine kopyalandı.


In [ ]:
import cv2
import os
from pathlib import Path

# Ana ayarlar
root_dir =  r"C:\Users\Admin\Desktop\TEZ\Deepfake"
frames_per_video = 30

# Frame çıkarma fonksiyonu
def extract_frames(video_path, output_folder, num_frames=30):
    cap = cv2.VideoCapture(str(video_path))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if total_frames < num_frames or total_frames == 0:
        frame_indices = list(range(min(total_frames, num_frames)))
    else:
        interval = total_frames // num_frames
        frame_indices = [i * interval for i in range(num_frames)]

    video_name = video_path.stem
    count = 0
    for i in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if ret:
            frame_name = f"{video_name}_frame_{count:03d}.jpg"
            cv2.imwrite(str(output_folder / frame_name), frame)
            count += 1
    cap.release()

# İşlenecek klasör ve hedef klasör isimleri
split_label_dirs = {
    ("train", "real"): "train/train-real-frame",
    ("train", "fake"): "train/train-fake-frame",
    ("test", "real"): "test/test-real-frame",
    ("test", "fake"): "test/test-fake-frame"
}

# Tüm videoları işleyip frameleri çıkart
for (split, label), output_rel in split_label_dirs.items():
    video_dir = Path(root_dir) / split / label
    output_dir = Path(root_dir) / output_rel
    output_dir.mkdir(parents=True, exist_ok=True)

    for video_file in video_dir.glob("*.*"):
        if video_file.suffix.lower() in [".mp4", ".avi", ".mov", ".mkv"]:
            extract_frames(video_file, output_dir, frames_per_video)

print("✅ Frame çıkarma işlemi tamamlandı.")


In [1]:
pip install albumentations opencv-python tqdm

  Using cached albumentations-2.0.5-py3-none-any.whl.metadata (41 kB)
  Using cached pydantic-2.11.3-py3-none-any.whl.metadata (65 kB)
  Using cached albucore-0.0.23-py3-none-any.whl.metadata (5.3 kB)
  Using cached opencv_python_headless-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached albumentations-2.0.5-py3-none-any.whl (290 kB)
Using cached albucore-0.0.23-py3-none-any.whl (14 kB)
Using cached opencv_python_headless-4.11.0.86-cp37-abi3-win_amd64.whl (39.4 MB)
Using cached pydantic-2.11.3-py3-none-any.whl (443 kB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.8.2
    Uninstalling pydantic-2.8.2:
      Successfully uninstalled pydantic-2.8.2
Note: you may need to restart the kernel to use updated packages.


🚀 Augmenting train-real-frame: 21331 → 135330 (eklenecek: 113999)
✅ 113999 yeni frame oluşturuldu: C:\Users\Admin\Desktop\TEZ\Deepfake\DeepFake-train-test\train\train-real-frame-aug
🚀 Augmenting test-real-frame: 5340 → 33840 (eklenecek: 28500)
✅ 28500 yeni frame oluşturuldu: C:\Users\Admin\Desktop\TEZ\Deepfake\DeepFake-train-test\test\test-real-frame-aug
